# Load packages

In [1]:
from core.DataLoader import *
from core.models.MLP import * 
from core.models.GAM import *
from core.models.SVM import *
from core.models.RandomForest import *
from core.models.Classifier import *
from core.models.XGB import *

In [2]:
clf_model_list ={
    "MLP": MLPClassifier,
    "DT": m_DecisionTreeClassifier,
    "RF": m_RandomForestClassifier,
    "LR": m_LogisticRegression,
    "SVM": SVMClassifier,
    "XGB": XGBOOSTClassifier
}
clf_model_name_list = ['MLP', 'RF(5)', 'RF(U)', 'DT(5)', 'DT(U)', 'LR', 'SVM', 'XGB']

# Load training data

In [3]:
engine = 'postgres'# use 'postgres' or 'mssql'
dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', datasets=['ssb', 'tpch', 'tpch_10', 'tpch_100', 'imdb'], suffix=None)
features = dl.base_features
print("Using features: ", features)

Using features:  ['left_cardinality', 'base_cardinality', 'sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'sel_of_join_pred', 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed', 'result_size', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr']


# Train and collect model accuracy

In [4]:
model_accs = {
    'MLP': [],
    'RF(5)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}

present_idxs = range(len(one_file_dss))

for i in present_idxs:  # range(len(one_file_dss)):

    print(f"Processing binary join # {i}")
    ds = one_file_dss[i]
    ds_name = one_file_names[i]
    
    # ======================= Train test split =========================
    X = ds[features]
    y = ds['optimal_decision']
    X_costs = ds[dl.regression_targets]

    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
    X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
    X_train, X_test, y_train, y_test = \
        X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()
    
    # Calculate sample weights
    X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
    # ======================= End data splitting =========================
    
    for idx, model_name in enumerate(clf_model_name_list):
        model = clf_model_list[model_name.split('(')[0]]        
        if 'DT' in model_name or 'RF' in model_name:
            max_depth = model_name.split('(')[-1].split(')')[0]
            if max_depth.lower() == 'u':
                max_depth = None
            else:
                max_depth = int(max_depth)
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights, max_depth=max_depth)
        elif model_name == 'MLP':
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights, max_iter=500, weight_decay=0.000001)
        else:
            clf = model().fit(X_train, y_train, sample_weight=X_train_weights)
            
        print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
        
        model_accs[model_name].append(clf.score(X_test, y_test))
    

  0%|          | 0/500 [00:00<?, ?it/s]

Processing binary join # 0
set features: 12


100%|██████████| 500/500 [00:55<00:00,  9.05it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.975
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9425
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.97
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9425
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.995
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.9475
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.91


  1%|          | 3/500 [00:00<00:22, 22.54it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.98
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.9625
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.975
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9625
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.995
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.955
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.945


  1%|          | 3/500 [00:00<00:18, 26.47it/s]

Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 0.97
Processing binary join # 62
set features: 12


100%|██████████| 500/500 [00:21<00:00, 22.79it/s]


Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9775
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.935
Accuray of <class 'core.models.RandomForest.m_RandomForestClassifier'>: 0.97
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.9625
Accuray of <class 'core.models.RandomForest.m_DecisionTreeClassifier'>: 0.995
Accuray of <class 'core.models.GAM.m_LogisticRegression'>: 0.9625
Accuray of <class 'core.models.SVM.SVMClassifier'>: 0.9425
Accuray of <class 'core.models.XGB.XGBOOSTClassifier'>: 0.9725


# Model Accuracy

In [6]:
import pprint
pprint.pprint(model_accs)

{'DT(5)': [0.9425,
           1.0,
           0.9725,
           0.97,
           0.9975,
           1.0,
           1.0,
           1.0,
           0.945,
           0.925,
           0.9475,
           0.9575,
           0.9425,
           1.0,
           1.0,
           1.0,
           0.995,
           0.995,
           1.0,
           0.9925,
           1.0,
           0.995,
           1.0,
           0.965,
           0.895,
           0.975,
           0.975,
           0.985,
           0.9725,
           0.98,
           1.0,
           0.9725,
           0.9675,
           1.0,
           0.9975,
           0.9925,
           0.99,
           1.0,
           0.94,
           0.9725,
           0.9875,
           0.955,
           0.9725,
           1.0,
           0.9975,
           0.9875,
           0.97,
           0.955,
           1.0,
           1.0,
           0.99,
           1.0,
           1.0,
           0.96,
           0.9175,
           0.99,
           0.98,
 